In [6]:
##7.1투표기반분류기##
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, Y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, Y_train)

#개별분류기의 예측을 평균내어 확률이 가장 높은 클래스예측
#확률을 추정하기위해 교차검증 사용, 훈련속도 느리지만 정확함
voting_proba_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')
voting_proba_clf.fit(X_train, Y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomF...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [8]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    #각 분류기의 테스트셋 정확도 출력
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(Y_test, Y_pred))
    
#각 분류기의 개별성능보다 투표기반분류기의 성능이 더 높다

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.888
VotingClassifier 0.904


In [10]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_proba_clf):
    #각 분류기의 테스트셋 정확도 출력
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(Y_test, Y_pred))
    
#각 분류기의 개별성능보다 투표기반분류기의 성능이 더 높다

LogisticRegression 0.864
RandomForestClassifier 0.888
SVC 0.888
VotingClassifier 0.928


In [15]:
##7.2.1 배깅##
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1) #페이스팅을 사용하려면 bootstrap=False
bag_clf.fit(X_train, Y_train)
Y_pred = bag_clf.predict(X_test)

print('Accuracy =', accuracy_score(Y_test, Y_pred))
#샘플을 중복선택하면 배깅, 이미 뽑은 샘플은 선택하지 않으면 페이스팅

Accuracy = 0.92


In [16]:
##7.2.2 oob 평가##
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True) #선택되지 않은 샘플로 평가

bag_clf.fit(X_train, Y_train)
bag_clf.oob_score_

0.9013333333333333

In [17]:
from sklearn.metrics import accuracy_score

Y_pred = bag_clf.predict(X_test)
accuracy_score(Y_test, Y_pred)

0.904

In [18]:
bag_clf.oob_decision_function_ #oob샘플에 대한 결정함수의 값, 확률값으로 나타남

array([[0.39893617, 0.60106383],
       [0.4198895 , 0.5801105 ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.06701031, 0.93298969],
       [0.36702128, 0.63297872],
       [0.01744186, 0.98255814],
       [0.97014925, 0.02985075],
       [0.9744898 , 0.0255102 ],
       [0.79661017, 0.20338983],
       [0.00512821, 0.99487179],
       [0.78947368, 0.21052632],
       [0.79213483, 0.20786517],
       [0.96984925, 0.03015075],
       [0.05555556, 0.94444444],
       [0.        , 1.        ],
       [0.99014778, 0.00985222],
       [0.90625   , 0.09375   ],
       [0.99404762, 0.00595238],
       [0.01554404, 0.98445596],
       [0.34636872, 0.65363128],
       [0.90816327, 0.09183673],
       [1.        , 0.        ],
       [0.96721311, 0.03278689],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.6284153 , 0.3715847 ],
       [0.